In [1]:
import numpy as np
import itertools

def generate_configurations(N):
    """Generate all possible spin configurations for N spins."""
    return list(itertools.product([-1, 1], repeat=N))

def calculate_energy(configuration, L, J):
    """Calculate the energy of a given spin configuration."""
    energy = 0
    for i in range(L):
        for j in range(L):
            spin = configuration[i * L + j]
            # Interactions with right and down neighbors (periodic boundary conditions)
            right = configuration[i * L + (j + 1) % L]
            down = configuration[((i + 1) % L) * L + j]
            energy -= J * spin * (right + down)
    return energy

def calculate_probabilities(L, J, T):
    """Calculate the probability vector of all configurations at temperature T."""
    N = L * L
    beta = 1.0 / T
    configurations = generate_configurations(N)
    energies = np.array([calculate_energy(config, L, J) for config in configurations])
    boltzmann_factors = np.exp(-beta * energies)
    Z = np.sum(boltzmann_factors)
    probabilities = boltzmann_factors / Z
    return configurations, probabilities

# Parameters
L = 2  # Lattice dimension (LxL)
J = 1.0  # Interaction strength
T = 1e-1  # Temperature

# Calculate probabilities
configs, probs = calculate_probabilities(L, J, T)

# Display results
for config, prob in zip(configs, probs):
    print(f"Configuration: {config}, Probability: {prob:.6f}")

print(f"probs={probs}")
print(f"Entropy is {-np.sum(probs*np.log(probs))}")


Configuration: (-1, -1, -1, -1), Probability: 0.500000
Configuration: (-1, -1, -1, 1), Probability: 0.000000
Configuration: (-1, -1, 1, -1), Probability: 0.000000
Configuration: (-1, -1, 1, 1), Probability: 0.000000
Configuration: (-1, 1, -1, -1), Probability: 0.000000
Configuration: (-1, 1, -1, 1), Probability: 0.000000
Configuration: (-1, 1, 1, -1), Probability: 0.000000
Configuration: (-1, 1, 1, 1), Probability: 0.000000
Configuration: (1, -1, -1, -1), Probability: 0.000000
Configuration: (1, -1, -1, 1), Probability: 0.000000
Configuration: (1, -1, 1, -1), Probability: 0.000000
Configuration: (1, -1, 1, 1), Probability: 0.000000
Configuration: (1, 1, -1, -1), Probability: 0.000000
Configuration: (1, 1, -1, 1), Probability: 0.000000
Configuration: (1, 1, 1, -1), Probability: 0.000000
Configuration: (1, 1, 1, 1), Probability: 0.500000
probs=[5.00000000e-01 9.02425694e-36 9.02425694e-36 9.02425694e-36
 9.02425694e-36 9.02425694e-36 1.62874427e-70 9.02425694e-36
 9.02425694e-36 1.628744